### Preprocessing
224x224 size로 비율을 유지하면서 Resize한다

In [2]:
import os
train_path = os.getcwd()+"/Dataset/train/"
valid_path = os.getcwd()+"/Dataset/valid/"
origin_path = os.getcwd()+"/Dataset/imagenet/imagenet_images/"
class_list= os.listdir(origin_path)

In [3]:
import numpy as np
from PIL import Image
def preprocess(origin_path, new_train_path, new_valid_path):
    ext=origin_path.split(".")[-1]
    if not ext in ["jpg","png"]:
        return
    img = Image.open(origin_path)
    width,height = img.size
    tmp=img.resize((width*256//height,256))
    width, height = tmp.size
    new_img = Image.new("RGB", (256,256))   
    new_img.paste(tmp, ((256-width)//2,(256-height)//2))
    if np.random.random(1)[0]>0.75:
        new_img.save(new_train_path)
    else:
        new_img.save(new_valid_path)
    img.close()
    del img,tmp,new_img

In [6]:
from multiprocessing import Pool
folder_list = os.listdir(origin_path)
class_done =0
for item in folder_list:
    os.makedirs(train_path+item,exist_ok=True)
    os.makedirs(valid_path+item,exist_ok=True)
    img_folder_path = origin_path+item+"/"
    img_list = os.listdir(img_folder_path)
    param_list = list(map( lambda x :
        (img_folder_path+x,train_path+item+"/"+x,valid_path+item+"/"+x),img_list))
    proc=Pool(os.cpu_count()-2)
    proc.starmap(preprocess, param_list)
    class_done+=1
print(f"class {class_done}개 완료!")

/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1441814 bytes but only got 0. Skipping tag 33
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 428 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 196608 bytes but only got 0. Skipping tag 60
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 196608 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5

class 973개 완료!


Process ForkPoolWorker-29177:
Process ForkPoolWorker-29181:
Process ForkPoolWorker-29179:
Process ForkPoolWorker-29174:
Process ForkPoolWorker-29169:
Process ForkPoolWorker-29166:
Process ForkPoolWorker-29163:
Process ForkPoolWorker-29178:
Process ForkPoolWorker-29161:
Process ForkPoolWorker-29170:
Process ForkPoolWorker-29172:
Process ForkPoolWorker-29180:
Process ForkPoolWorker-29188:
Process ForkPoolWorker-29171:
Process ForkPoolWorker-29173:
Process ForkPoolWorker-29190:
Process ForkPoolWorker-29164:
Process ForkPoolWorker-29167:
Process ForkPoolWorker-29189:
Process ForkPoolWorker-29176:
Process ForkPoolWorker-29165:
Process ForkPoolWorker-29175:
Process ForkPoolWorker-29168:
Process ForkPoolWorker-29162:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


### Network
AlexNet의 구조를 가져왔습니다.

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch import flatten

#CNN input:(224,224,3)
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            #1
            nn.Conv2d(3, 96, kernel_size=11, stride=4,padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
#             nn.LocalResponseNorm(size=5,k=2),
            #2
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
#             nn.LocalResponseNorm(size=5,k=2),
            #3
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #4
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #5
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1000)
        )

    def forward(self, x):
        x = self.features(x)
        x = flatten(x, 1)
        x = self.classifier(x)
        x = F.softmax(x,dim=1)
        return x

### Train
Valid의 계산의 경우, FiveCrop을 위해 pytorch documentation을 참조했습니다.

In [ ]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import numpy as np

from torch import save,stack
from torch.utils.tensorboard import SummaryWriter

import torch.nn as nn

net=nn.DataParallel(AlexNet()).cuda()

learning_rate = 1e-3
num_epoch = 150
batch_size =384
summary = SummaryWriter(log_dir="runs/AlexNet01")

trn_dataset = ImageFolder(train_path, transform=transforms.Compose([
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()]))
trn_size=len(trn_dataset)
trn_loader= DataLoader(trn_dataset, batch_size = batch_size, 
                       shuffle= True, num_workers=4)
val_dataset = ImageFolder(valid_path, transform=transforms.Compose([
    transforms.FiveCrop(224),
    transforms.Lambda(lambda crops: stack([transforms.ToTensor()(crop) for crop in crops]))
]))
val_size=len(val_dataset)
val_loader= DataLoader(val_dataset, batch_size = batch_size, 
                       shuffle= True, num_workers=4)

criterion = CrossEntropyLoss()
optimizer = Adam(net.parameters(),lr = learning_rate)

best_val_loss = float('inf')
for epoch in range(num_epoch):
    trn_loss = 0
    trn_acc =0
    net.train()
    for _, (x,label) in enumerate(trn_loader):
        x, label = x.cuda(), label.cuda()
        optimizer.zero_grad()
        pred = net(x)
        loss = criterion(pred, label)
        loss.backward()
        optimizer.step()
        
        trn_loss += loss.item()
        pred,label = pred.cpu().detach().numpy(),label.cpu().detach().numpy()
        trn_acc += np.sum(np.argmax(pred,axis=1)==label)
        # del(memory saved)
        del loss,pred
        
    trn_loss/=len(trn_loader)
    trn_acc/=trn_size
    
    val_loss= 0
    val_acc=0
    net.eval()
    for _, (x,label) in enumerate(val_loader):
#         x, target = x # input is a 5d tensor, target is 2d
        x, label = x.cuda(), label.cuda()
        bs, ncrops, c, h, w = x.size()
        pred = net(x.view(-1, c, h, w)) # fuse batch size and ncrops
        pred = pred.view(bs, ncrops, -1).mean(1) # avg over crops
        loss = criterion(pred, label)
        
        val_loss += loss.item()
        pred,label = pred.cpu().detach().numpy(),label.cpu().detach().numpy()
        val_acc += np.sum(np.argmax(pred,axis=1)==label)
        del loss,pred
    val_loss/=len(val_loader)
    val_acc/=val_size
    print("epoch: {}/{} | trn loss: {:.4f} | val loss: {:.4f} | trn_acc: {:.3f} | val_acc: {:.3f}".format(
                epoch+1, num_epoch, trn_loss, val_loss, trn_acc, val_acc))
    if val_loss<best_val_loss:
        save(net.state_dict(),'./Model/AlexNet.pth')
    summary.add_scalars('loss',{'valid':val_loss, 'train':trn_loss},epoch)    
    summary.add_scalars('acc',{'valid':val_acc, 'train':trn_acc},epoch)
